In [2]:
cd /home/rh/Arbeitsraum/bitbucket/share/SLMI2

/home/rh/Arbeitsraum/bitbucket/share/SLMI2


In [3]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import datetime as datetime
import os
from IPython.display import display, HTML
from math import isclose
from collections import Counter
import multiprocessing

# Network libary
from graph_tool.all import *
import networkx as nx
import networkit as nk

# sklearn
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

In [6]:
df_node2cnt

,0,1
0,115,16
1,170,10
2,124,2
3,39,5
4,111,1
...,...,...
177,43,1
178,175,1
179,182,1
180,1,1


In [7]:
import pandas as pd
import numpy as np
loaded = np.load("./tables/enron_staticlinkpred.npz",allow_pickle=True)
print(loaded.files)

edges_train = loaded["arr_0"]
labels_train = loaded["arr_1"]
edges_valid = loaded["arr_2"]
labels_valid = loaded["arr_3"]
edges_test = loaded["arr_4"]
labels_test = loaded["arr_5"]
df_node2cnt = pd.DataFrame(loaded["arr_6"])
df_node2cnt.columns = ["node","cnt"]
df_train_0 = pd.DataFrame(edges_train)
df_train_0[2] = labels_train
cond = df_train_0[2] == 1
df_train_1 = df_train_0.loc[cond].copy()
df_train = df_train_1[[0,1]].copy()
df_train.columns = ["source","target"]
df_train_0.columns = ["source","target","label"]

df_valid_0 = pd.DataFrame(edges_valid)
df_valid_0[2] = labels_valid
cond = df_valid_0[2] == 1
df_valid_1 = df_valid_0.loc[cond].copy()
df_valid = df_valid_1[[0,1]].copy()
df_valid.columns = ["source","target"]
df_valid_0.columns = ["source","target","label"]


df_test_0 = pd.DataFrame(edges_test)
df_test_0[2] = labels_test
cond = df_test_0[2] == 1
df_test_1 = df_test_0.loc[cond].copy()
df_test = df_test_1[[0,1]].copy()
df_test.columns = ["source","target"]
df_test_0.columns = ["source","target","label"]


['arr_0', 'arr_1', 'arr_2', 'arr_3', 'arr_4', 'arr_5', 'arr_6']


In [8]:
out_tr,out_te = [],[]
    
uni_nodes = list(set(df_train["source"]))
uni_nodes.extend(list(set(df_train["target"])))
uni_nodes = list(set(uni_nodes))
    
node2index = dict()
for i in range(len(uni_nodes)):
    node2index.update({uni_nodes[i]:i})

for i in range(len(df_train)):
    source_index = node2index[df_train["source"].iloc[i]]
    target_index = node2index[df_train["target"].iloc[i]]
    out_tr.append([source_index,target_index])
        
for i in range(len(df_test)):
    source_index = node2index[df_test["source"].iloc[i]]
    target_index = node2index[df_test["target"].iloc[i]]
    out_te.append([source_index,target_index])
    
df_train_index = pd.DataFrame(out_tr)
df_test_index = pd.DataFrame(out_te)
df_train_index.columns = ["source","target"]
df_test_index.columns = ["source","target"]
#df_train_index.to_csv("/home/rh/law_df_train.csv",index=False,header=None)
#df_test_index.to_csv("/home/rh/law_df_test.csv",index=False,header=None)

In [9]:
import networkx as nx
import matplotlib.pyplot as plt
G = nx.Graph()
#G.add_edges_from([(1, 2), (1, 3), (1, 4), (3, 4), (4, 5)])

In [11]:
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import average_precision_score, precision_recall_curve
#from sklearn.metrics import plot_precision_recall_curve


out_tr = []
for i in range(len(df_train_index)):
    s = df_train_index["source"].iloc[i]
    t = df_train_index["target"].iloc[i]
    out_tr.append((s, t))
    
out_te = []
for i in range(len(edges_test)):
    s = node2index[edges_test[i][0]]
    t = node2index[edges_test[i][1]]
    out_te.append((s, t))
    
G = nx.Graph()
G.add_edges_from(out_tr)


In [12]:
for kkk in range(1,5):
    # 1:"adamic", 2:"resource",3:"jaccard",4:"preferential"
    model_type = kkk

    if model_type == 1:
        preds = nx.adamic_adar_index(G, out_te)
        model_name = "adamic"
    elif model_type == 2:
        preds = nx.resource_allocation_index(G,out_te)
        model_name = "resource allocation"
    elif model_type == 3:
        preds = nx.jaccard_coefficient(G, out_te)
        model_name = "Jaccard Coefficient"
    elif model_type==4:
        preds = nx.preferential_attachment(G,out_te)
        model_name = "Preferential"
    prediction = []
    for u, v, p in preds:
        prediction.append(p)

    print(model_name)
    fpr, tpr, thresholds = metrics.roc_curve(labels_test, prediction)
    auc = metrics.auc(fpr, tpr)
    print(auc)
    # Average precision score
    average_precision = average_precision_score(labels_test, prediction)
    print(average_precision)
    # Data to plot precision - recall curve
    precision, recall, thresholds = precision_recall_curve(labels_test, prediction)
    # Use AUC function to calculate the area under the curve of precision recall curve
    auc_precision_recall = metrics.auc(recall, precision)
    print(auc_precision_recall)
    print("====")

adamic
0.8725212208981012
0.6298439961706144
0.6324371334222672
====
resource allocation
0.8795738000013534
0.6584542235082396
0.6612343525351427
====
Jaccard Coefficient
0.8250378400813871
0.5308215781949049
0.5362246102291636
====
Preferential
0.7739362482038651
0.46132497838955255
0.4621871803107412
====
